In [119]:
%load_ext autoreload
%autoreload 2
import os
import dotenv
from stravaio import StravaIO
import numpy as np
import pandas as pd
dotenv.load_dotenv()
access_token = os.getenv('ACCESS_TOKEN')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from plotly.offline import iplot, init_notebook_mode
import plotly
import plotly.graph_objs as go
import plotly_express as px
# init_notebook_mode(connected=True)

In [5]:
plotly.__version__

'3.10.0'

In [3]:
# Some constants to be used throughout the calculations
ATHLETE_ID = 1202065

In [7]:
client = StravaIO(access_token)

# Get information about the athlete

## Option 1:  Get the athlete info from strava

In [18]:
athlete = client.get_logged_in_athlete()
athlete

{
  "id": 1202065,
  "resource_state": 3,
  "firstname": "Maksym",
  "lastname": "Sladkov",
  "profile_medium": "https://dgalywyr863hv.cloudfront.net/pictures/athletes/1202065/329723/1/medium.jpg",
  "profile": "https://dgalywyr863hv.cloudfront.net/pictures/athletes/1202065/329723/1/large.jpg",
  "city": "Graz",
  "state": "Styria",
  "country": "Austria",
  "sex": "M",
  "friend": null,
  "follower": null,
  "premium": false,
  "summit": false,
  "created_at": "2012-10-13T14:40:18Z",
  "updated_at": "2019-02-16T16:17:09Z",
  "follower_count": 292,
  "friend_count": 112,
  "mutual_friend_count": 0,
  "measurement_preference": "meters",
  "ftp": 240,
  "weight": 73.102,
  "clubs": [
    {
      "id": 22072,
      "resource_state": 2,
      "name": "Racefietsblog.nl",
      "profile_medium": "https://dgalywyr863hv.cloudfront.net/pictures/clubs/22072/479694/5/medium.jpg",
      "cover_photo": "https://dgalywyr863hv.cloudfront.net/pictures/clubs/22072/4996645/7/large.jpg",
      "cover_pho

## Option 2: Get the athlete from local storage (assuming at was already stored)

In [22]:
# A generator with a single element will be returned, access the element by calling next()..,
athlete = next(client.local_athletes())
assert athlete['id']==ATHLETE_ID

# Load athlete activities (last year)

## Option 1: load activities from Strava

In [12]:
# Returns a list of summary activities
activities = client.get_logged_in_athlete_activities(after='last week')

Fetched 3, the latests is on 2019-06-09 11:13:18+00:00


In [13]:
[a.name for a in activities]

['G.O.A.T', 'With Antonio', 'Afternoon Ride']

In [14]:
a = client.get_activity_by_id(id=activities[-1].id)

In [15]:
a

Activity: 2436304110, Date: 2019-06-09 11:13:18+00:00, Name: Afternoon Ride

In [38]:
s = client.get_activity_streams(activities[-1].id, athlete_id=athlete.id)
s

Streams for 2436304110
Keys: ['time', 'distance', 'altitude', 'velocity_smooth', 'heartrate', 'cadence', 'watts', 'temp', 'moving', 'grade_smooth', 'lat', 'lng']
Access: obj.key

In [39]:
df = pd.DataFrame(s.to_dict())

In [40]:
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng
0,165,458.7,389.6,5.0,113,82,131,26,True,3.2,47.084871,15.442007
1,166,463.3,389.8,4.9,113,80,165,26,True,3.4,47.084894,15.442057
2,167,467.7,390.0,4.8,113,78,168,26,True,4.7,47.084916,15.442106
3,168,471.7,390.2,4.5,113,77,178,26,True,3.6,47.084932,15.442152
4,169,475.8,390.4,4.3,113,78,177,26,True,2.5,47.084949,15.442200


In [104]:
ftp=257
strava_orange = '#fc4c02'
df['color']= 'white'
df.loc[df['watts'] > ftp*0.6, 'color'] = 'grey'
df.loc[df['watts'] > ftp*0.9, 'color'] = strava_orange
df[df['watts']==0] = None
df = df.dropna()

In [105]:
df.head()

,time,distance,altitude,velocity_smooth,heartrate,cadence,watts,temp,moving,grade_smooth,lat,lng,color
0,165.0,458.7,389.6,5.0,113.0,82.0,131.0,26.0,1.0,3.2,47.084871,15.442007,white
1,166.0,463.3,389.8,4.9,113.0,80.0,165.0,26.0,1.0,3.4,47.084894,15.442057,grey
2,167.0,467.7,390.0,4.8,113.0,78.0,168.0,26.0,1.0,4.7,47.084916,15.442106,grey
3,168.0,471.7,390.2,4.5,113.0,77.0,178.0,26.0,1.0,3.6,47.084932,15.442152,grey
4,169.0,475.8,390.4,4.3,113.0,78.0,177.0,26.0,1.0,2.5,47.084949,15.442200,grey


In [132]:
out, bins = pd.cut(df.watts, bins=np.arange(0, df.watts.max(), 25), retbins=True)

In [133]:
bins

array([  0.,  25.,  50.,  75., 100., 125., 150., 175., 200., 225., 250.,
       275., 300., 325., 350., 375., 400., 425., 450.])

In [140]:
hist = df['watts'].groupby(out).count()

In [159]:
bin_c = pd.DataFrame({'watts': bins})

In [160]:
bin_c['color'] = 'white'

In [164]:
bin_c.loc[bin_c['watts'] > 0.6*ftp, 'color'] = 'grey'
bin_c.loc[bin_c['watts'] > 0.9*ftp, 'color'] = strava_orange

In [177]:
altitude = go.Scatter(
    x=df.time,
    y=df.altitude,
    fill='tozeroy',
    fillcolor='rgb(30,30,30)',
    line=dict(
        color='rgb(30,30,30)'
    )
)

watts = go.Scattergl(
    x=df.time,
    y=df.watts,
    mode='markers',
    marker=dict(
        color=df['color'],
        size=2
    )
)


watts_b = go.Bar(
    y = bins,
    x = hist,
    orientation='h',
    marker=dict(
        color = bin_c['color']
    )
)



fig = plotly.tools.make_subplots(rows=1, cols=2, shared_yaxes=True)

fig.append_trace(altitude, 1,1)
fig.append_trace(watts, 1,1)
fig.append_trace(watts_b, 1,2)
fig.layout.update(plot_bgcolor='black')
fig.layout.update(xaxis=dict(domain=[0, 0.8]))
fig.layout.update(xaxis2=dict(domain=[0.8, 1.0]))
fig.layout.update(bargap=0)

iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [108]:
watts_h

Histogram({
    'marker': {'color': array(['white', 'grey', 'grey', ..., 'white', '#fc4c02', '#fc4c02'],
         dtype=object)},
    'y': array([131., 165., 168., ...,  20., 323., 406.])
})

In [67]:
df.color

'orange'